## How to get this data

In [1]:
def get_opening_hours_from_osm_data(path_to_osm_geojson):
    """
        path_to_osm_geojson (geojson): path to the geojson file. We get data from https://download.bbbike.org/osm/bbbike/ 
    """

    f = open(path_to_osm_geojson, "r")
    all_lines = f.readlines()
    print("Number of lines in file:", len(all_lines))

    line_index = 0
    opening_hours_indexes = []
    
    f = open(path_to_osm_geojson, "r")
    for x in f:
        if 'opening_hours' in x:
            opening_hours_indexes.append(line_index)
        line_index += 1
    print("Number of opening_hours entries:", len(opening_hours_indexes))
    return all_lines, opening_hours_indexes


def write_opening_hours_to_geojson(path_to_output_geojson, all_lines, opening_hours_indexes):
    """
        all_lines (list): all lines in the overall file
        
    """
    with open(path_to_output_geojson, "w+") as my_file:
        my_file.write(all_lines[0])
        for line in opening_hours_indexes[:-1]:
            my_file.write(all_lines[line])
        # remove comma from final line
        final_line = all_lines[opening_hours_indexes[-1]].replace(",\n","\n")
        my_file.write(final_line)
        my_file.write(all_lines[-1])
        my_file.close()


In [2]:
# all_lines, opening_hours_indexes = get_opening_hours_from_osm_data(path_to_osm_geojson="../Data/shapes/Montreal.osm.geojson")
# write_opening_hours_to_geojson(all_lines=all_lines, opening_hours_indexes=opening_hours_indexes, path_to_output_geojson="../Data/model_inputs/mtl_opening_hours.geojson")
##then
# COLUMNS_TO_USE = ['name','opening_hours','amenity','tourism', 'shop', 'healthcare', 'leisure', 'sport','craft', 'building','geometry']
# all_opening_hours[COLUMNS_TO_USE].to_csv('../Data/model_inputs/mtl_opening_hours_cleaned.csv',index=False, encoding='utf-8')

# Opening Hour analysis

In [3]:
import geopandas as gpd
import pandas as pd
import shapely
import datetime
import re
import numpy as np

In [4]:
all_opening_hours = gpd.read_file("../../Data/model_inputs/mtl_opening_hours.geojson")

X_all = pd.read_csv('../../Data/model_inputs/gdf_2017_X.csv')
y_all = pd.read_csv('../../Data/model_inputs/gdf_2017_y.csv')


In [5]:
all_opening_hours = all_opening_hours.dropna(subset=['opening_hours'])

In [6]:
all_opening_hours.head()

,url,name,phone,name:en,tourism,wikidata,wikipedia,wheelchair,alt_name:en,alt_name:fr,...,u-pick,building:part,fuel:octane_92,fuel:octane_98,layer,lit,service:vehicle:tyres,service:vehicle:suspension,playground:theme,geometry
0,http://www.mbam.qc.ca,Musée des Beaux-Arts de Montréal,+1 514 285 2000,Montreal Museum of Fine Arts,museum,Q860812,en:Montreal Museum of Fine Arts,yes,Museum of Fine Arts,Musée des Beaux-Arts,...,None,None,None,None,None,None,None,None,None,POINT (-73.57940 45.49870)
1,None,Noodles Star,+1-514-932-2888,None,None,None,None,limited,None,None,...,None,None,None,None,None,None,None,None,None,POINT (-73.58042 45.49307)
2,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (-73.51726 45.49226)
3,None,Maxi,+1-450-672-3201,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (-73.46781 45.46025)
4,None,Ben & Florentine,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (-73.46731 45.46930)


In [48]:
all_opening_hours['amenity'].value_counts()

restaurant                354
cafe                      134
fast_food                  96
pharmacy                   59
bank                       55
bar                        41
library                    31
fuel                       28
pub                        18
bicycle_repair_station     16
clinic                     13
veterinary                 12
atm                        12
post_office                12
dentist                    11
ice_cream                  10
car_rental                 10
social_facility             8
parking                     7
charging_station            6
community_centre            6
bureau_de_change            6
marketplace                 4
driving_school              3
childcare                   3
toilets                     3
recycling                   3
shop|clothes                3
nightclub                   3
car_wash                    2
car_sharing                 2
food_court                  2
dojo                        2
theatre   

In [45]:
for col in all_opening_hours.columns:
    print(col)

url
name
phone
name:en
tourism
wikidata
wikipedia
wheelchair
alt_name:en
alt_name:fr
opening_hours
amenity
cuisine
smoking
capacity
addr:city
addr:street
addr:postcode
addr:province
addr:housenumber
status
barrier
designation
shop
website
atm
ref
brand
brand:wikidata
brand:wikipedia
addr:suburb
internet_access
internet_access:fee
operator
dispensing
drive_through
wikipedia:fr
door
email
entrance
automatic_door
takeaway
outdoor_seating
level
name:fr
alt_name
addr:door
payment:cash
toilets:wheelchair
payment:debit_cards
payment:credit_cards
internet_access:operator
delivery
brand:en
brand:fr
official_name
official_name:en
official_name:fr
source
healthcare
leisure
organic
contact:email
contact:phone
contact:website
payment:visa
payment:interac
payment:mastercard
payment:american_express
ele
addr:country
note
contact:twitter
payment:bitcoin
contact:facebook
internet_access:ssid
description
office
country
name:de
addr:unit
consulate
diplomatic
opening_hours:covid19
air_conditioning
gst_num

In [38]:
all_opening_hours['opening_hours']

0                               We-Su 10:00-17:00; Mo off
1                                       Mo-Su 11:00-23:00
2                                              6:00-22:00
3                                       Mo-Su 08:00-22:00
4                       Mo-Sa 06:00-15:00, Su 07:00-15:00
                              ...                        
2044                 Mo-Fr 09:00-18:00; Sa-Su 09:00-17:00
2045                    Mo-Fr 08:30-18:00; Sa 08:00-13:00
2046    Apr-Nov: Th-Tu 08:00-17:00;Nov-Apr: Su 08:00-1...
2047                                           6:00-22:00
2048                                           6:00-22:00
Name: opening_hours, Length: 2042, dtype: object

In [8]:
all_opening_hours['shop'].value_counts()

supermarket                  98
convenience                  72
clothes                      65
hairdresser                  46
bakery                       39
                             ..
cosmetics                     1
bicycle,_scooter,_rentals     1
anime                         1
car_parts                     1
carpet                        1
Name: shop, Length: 107, dtype: int64

In [9]:
def change_crs_of_X(X, crs_from="EPSG:4236",crs_to="EPSG:3347"):
    """
        Function for translating the data into Canada Lambert projection 
        EPSG: 3347, so that the base unit is 1 m
    """
    new_X = X.copy()
    new_X['geometry'] = new_X.apply(lambda row: shapely.geometry.Point(row['long'],row['lat']),axis=1)
    new_X = gpd.GeoDataFrame(new_X,crs=crs_from)
    new_X = new_X.to_crs(crs_to)
    return new_X

In [10]:
## append end time to trips
start_end_times = gpd.read_file('../../Data/mtl_trajet/mtl_trajet_2017_final.shp')[['id_trip','starttime','endtime']]
X_all = X_all.merge(start_end_times, on='id_trip')

In [11]:
buffer_size_m = 50
geo_X = change_crs_of_X(X_all)

geo_X['buffers'] = geo_X['geometry'].apply(lambda row: row.buffer(buffer_size_m))

# get a geo-dataframe with only the Trip ID,trip end buffer and trip purpose    
only_buffers = geo_X[['id_trip','endtime','buffers']]
only_buffers =  gpd.GeoDataFrame(only_buffers.rename(columns={'buffers':'geometry'}), crs="EPSG:3347")

In [12]:
all_opening_hours = gpd.GeoDataFrame(all_opening_hours, crs='EPSG:4326')
all_opening_hours = all_opening_hours.to_crs('EPSG:3347')

In [13]:
def get_prop_open_hours(buffers, opening_hours):
    joined_data = gpd.sjoin(buffers, opening_hours, op='intersects', how='left')
    return joined_data

In [14]:
joined_data = get_prop_open_hours(only_buffers, all_opening_hours)

In [15]:
joined_data = joined_data.dropna(subset=['opening_hours'])

In [16]:
joined_data['endtime'] = pd.to_datetime(joined_data['endtime'])

In [17]:
IS_OPEN = ["24/7", "All Day, 7/24", "7/24"]
DAYOFWEEK_NAMES = {0:"Mo", 1:"Tu", 2:"We", 3:"Th", 4:"Fr", 5:"Sa", 6:"Su"}
MONTHS = ["Jan","Feb","Mar", "Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
LIST_DAYS = list(DAYOFWEEK_NAMES.values())
# EDGE CASES
# if no day
# Mo off; 
# Sa, Su off
# May-Dec
# 17:00+
# No time
# dim-lun
# until 22h
# 10:00AM to 5:00PM
# Mo-We 11:30-14:30, 17:30-22:30; Th 11:30-14:30, 17:30-23:30; Fr 11:30-14:30, 17:30-02:30; Sa 17:30-02:30; Su 17:30-22:30

In [18]:
datetime.datetime.strptime('03:55', '%H:%M').time() == datetime.datetime.strptime('03:55', '%H:%M').time()

True

In [19]:
def work_out_if_open(end_time, opening_time):
    day_of_week = DAYOFWEEK_NAMES[end_time.dayofweek]
    time_of_day = str(end_time.hour) + ':' + str(end_time.minute)
    
    ## check if , has been used instead of ; for splitting the opening times
    if len(re.findall('[0-9],', opening_time)) != 0:
        ## replace , with ; if so
        opening_time = change_comma_to_semicolon(opening_time)
        
    for opening_time_part in opening_time.split(';'):
        opening_time_part = opening_time_part.strip()
        # check if opening time string contains any days in it (will return True or False)
        all_days_not_in_opening_time = all([day not in opening_time_part for day in LIST_DAYS])
        ## 1. Check if just the time is provided or always open
        if all_days_not_in_opening_time:
            
            if opening_time_part in IS_OPEN:
                return True
            else:
                try:
                    bound_start, bound_end = extract_time(opening_time_part)
                except:
                    print('IGNORED', opening_time_part)
                    continue
                if within_opening_hours(bound_start, bound_end, time_of_day):
                    return True
                
        ## 2. Check day if open during
        all_days_in_opening_times = find_all_days_in_opening_time(opening_time_part)
        if day_of_week in all_days_in_opening_times:
            ## format string:
            opening_time_part = re.sub('[A-Z][a-z]-[A-Z][a-z]','',opening_time_part)
            opening_time_part = re.sub('[A-Z][a-z]','',opening_time_part)
            opening_time_part = opening_time_part.strip()
            try:
                bound_start, bound_end = extract_time(opening_time_part)
                if within_opening_hours(bound_start, bound_end, time_of_day):
                    return True
            except Exception as e:
                print(e, 'IGNORED', opening_time_part)
                continue
        else:
            continue

    return False


def change_comma_to_semicolon(opening_time):
    return re.sub('(?<=[0-9]),', ';', opening_time)
    

def extract_time(opening_time_part):
    split_parts = opening_time_part.split('-')
    ## Check there is two times
    try:
        assert len(split_parts) == 2
    except Exception as e:
        print(e)
    return split_parts



def find_all_days_in_opening_time(opening_time_part):
    all_days_found = []
    ## find all LIKE Mo-Fr and attach days
    part_of_str_to_ignore = []
    for day_range in re.finditer('[F-W][a-u]-[F-W][a-u]', opening_time_part):
        part_of_str_to_ignore.append(day_range.span())
        day_range = day_range.string.strip()
        try:
            start_range = key_from_value(DAYOFWEEK_NAMES, day_range[0:2])
            end_range = key_from_value(DAYOFWEEK_NAMES, day_range[3:5])
            if start_range == end_range:
                return list(DAYOFWEEK_NAMES.values())
            counter = 0
            while DAYOFWEEK_NAMES[(start_range+counter)%7] != DAYOFWEEK_NAMES[end_range]: 
                all_days_found.append(DAYOFWEEK_NAMES[(start_range+counter)%7])
                counter +=1
            all_days_found.append(DAYOFWEEK_NAMES[end_range])
            
        except Exception as e:
            print(e,  opening_time_part)
            continue
        
    ## make new string without day range
    part_of_str_to_ignore.sort()
    opening_time_part = remove_ignored_indexes(opening_time_part, part_of_str_to_ignore)
    all_days_found.extend(re.findall('[F-W][a-u]',opening_time_part))
    
    return all_days_found


def remove_ignored_indexes(orig_str, ignored_parts):
    if len(ignored_parts) == 0:
        return orig_str
    new_str = ""
    for i in range(len(ignored_parts)+1):
        if i == 0:
            new_str += orig_str[0:ignored_parts[i][0]]
        elif i == len(ignored_parts):
            new_str += orig_str[ignored_parts[i-1][1]:]
            break
        else:
            new_str += orig_str[ignored_parts[i-1][1]:ignored_parts[i][0]]
    return new_str
            
            
def key_from_value(the_dict, value):
    key = list(the_dict.keys())[list(the_dict.values()).index(value)]
    return key

def within_opening_hours(bound_start, bound_end, time_of_day):
    # if the opening time is 22:00 - 03:00 for example    
    if bound_start > bound_end:
        is_within = bound_start < time_of_day
    else:
        is_within = bound_start < time_of_day < bound_end
        
    return is_within

In [20]:
test_date = joined_data['endtime'].iloc[1]
test_date

Timestamp('2017-09-18 20:56:40')

In [21]:
test_ot = joined_data['opening_hours'].iloc[0]
test_ot

'Mo-Tu 09:00-18:00;We-Fr 09:00-21:00; Sa 10:00-17:00; Su 11:00-17:00'

In [22]:
joined_data['opened_at_time'] = joined_data.apply(lambda row: work_out_if_open(row['endtime'], row['opening_hours']), axis=1)


IGNORED PH closed

not enough values to unpack (expected 2, got 1) IGNORED off
'Ma' is not in list May-Dec Mo-We 09:00-18:00
'Ja' is not in list Jan-Apr We-Su 10:00-17:00

not enough values to unpack (expected 2, got 1) IGNORED 17:00+
' T' is not in list Su, Tu-Th 11:30-21:00

IGNORED PH closed

IGNORED dim-lun: 11:00 - 20:00 mar-sam: 10:00 - 23:00

too many values to unpack (expected 2) IGNORED -  09:00 - 22:00

too many values to unpack (expected 2) IGNORED -  09:00 - 22:00

IGNORED 7 days

IGNORED PH closed

IGNORED PH closed

IGNORED PH closed

IGNORED PH closed

IGNORED until 22h
'Ma' is not in list Ma-We 10:00-18:00

IGNORED dim-lun: 11:00 - 20:00 mar-sam: 10:00 - 23:00

IGNORED dim-lun: 11:00 - 20:00 mar-sam: 10:00 - 23:00

IGNORED PH closed

IGNORED Appointment

IGNORED "seasonal"

not enough values to unpack (expected 2, got 1) IGNORED off

IGNORED 7 days

not enough values to unpack (expected 2, got 1) IGNORED osed

IGNORED PH closed

IGNORED 7 days

too many values to unpac

not enough values to unpack (expected 2, got 1) IGNORED off

IGNORED PH closed

IGNORED PH closed
'Ma' is not in list May-Dec Mo-We 09:00-18:00
'Ja' is not in list Jan-Apr We-Su 10:00-17:00
'Ma' is not in list May-Dec Mo-We 09:00-18:00
'Ja' is not in list Jan-Apr We-Su 10:00-17:00
' T' is not in list Su, Tu-Th 16:00-22:00
' T' is not in list Su, Tu-Th 11:30-21:00

not enough values to unpack (expected 2, got 1) IGNORED off

not enough values to unpack (expected 2, got 1) IGNORED off

IGNORED PH closed

IGNORED "seasonal"

IGNORED PH closed

not enough values to unpack (expected 2, got 1) IGNORED off

IGNORED PH closed

not enough values to unpack (expected 2, got 1) IGNORED off

too many values to unpack (expected 2) IGNORED n-r:  16:00-23:00
'Ma' is not in list Mar-Dec: Mo-We 11:00-23:00

too many values to unpack (expected 2) IGNORED r-c:  15:00-23:00

not enough values to unpack (expected 2, got 1) IGNORED off
' T' is not in list Su, Tu-Th 11:30-21:00

IGNORED M-F: 07:30-21:00

too 

In [23]:
joined_data['opened_at_time'].value_counts()

False    10616
True     10420
Name: opened_at_time, dtype: int64

In [65]:
joined_data

,id_trip,endtime,geometry,index_right,url,name,phone,name:en,tourism,wikidata,...,u-pick,building:part,fuel:octane_92,fuel:octane_98,layer,lit,service:vehicle:tyres,service:vehicle:suspension,playground:theme,opened_at_time
8,142829,2017-09-18 17:36:34,"POLYGON ((7631406.916 1244778.181, 7631406.675...",163.0,None,Laprentania Florists,+1 514 288 9911,None,None,None,...,None,None,None,None,None,None,None,None,None,True
11,161711,2017-09-18 20:56:40,"POLYGON ((7628402.580 1240487.104, 7628402.339...",295.0,None,St-Viateur Bagel & Café,+1 514 487 8051,None,None,None,...,None,None,None,None,None,None,None,None,None,False
11,161711,2017-09-18 20:56:40,"POLYGON ((7628402.580 1240487.104, 7628402.339...",296.0,None,Bangkok Express,None,None,None,None,...,None,None,None,None,None,None,None,None,None,True
11,161711,2017-09-18 20:56:40,"POLYGON ((7628402.580 1240487.104, 7628402.339...",1122.0,None,Pâtisserie de Nancy,None,None,None,None,...,None,None,None,None,None,None,None,None,None,False
25,424306,2017-09-18 23:05:08,"POLYGON ((7632041.675 1246418.835, 7632041.435...",888.0,None,La graine brûlée,None,None,None,None,...,None,None,None,None,None,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25986,187909,2017-10-17 23:43:42,"POLYGON ((7628529.234 1247193.742, 7628528.993...",1884.0,None,Bibliothèque Marc-Favreau,+1-514-872-7272,None,None,None,...,None,None,None,None,None,None,None,None,None,False
25986,187909,2017-10-17 23:43:42,"POLYGON ((7628529.234 1247193.742, 7628528.993...",1889.0,None,Bibliothèque Marc-Favreau,+1-514-872-7272,None,None,None,...,None,None,None,None,None,None,None,None,None,False
25988,182050,2017-10-18 00:05:20,"POLYGON ((7630598.193 1245947.474, 7630597.953...",9.0,None,Schwartz's,+1-514-842-4813,None,None,Q3475576,...,None,None,None,None,None,None,None,None,None,False
25988,182050,2017-10-18 00:05:20,"POLYGON ((7630598.193 1245947.474, 7630597.953...",154.0,None,Prato Cafe Pizza,+1-514-285-1616,None,None,None,...,None,None,None,None,None,None,None,None,None,False


In [94]:
row[1]

id_trip                                                                  142829
endtime                                                     2017-09-18 17:36:34
geometry                      POLYGON ((7631406.915592751 1244778.181420075,...
index_right                                                               163.0
url                                                                        None
                                                    ...                        
lit                                                                        None
service:vehicle:tyres                                                      None
service:vehicle:suspension                                                 None
playground:theme                                                           None
opened_at_time                                                             True
Name: 8, Length: 338, dtype: object

In [97]:
for row in joined_data.iterrows():
    row_col = []
    for i, e in enumerate(joined_data.columns):
        if row[1][e]:
            row_col.append(e)

In [98]:
row_col

['id_trip',
 'endtime',
 'geometry',
 'index_right',
 'name',
 'name:en',
 'opening_hours',
 'amenity',
 'addr:city',
 'addr:street',
 'addr:postcode',
 'addr:province',
 'addr:housenumber',
 'website',
 'atm',
 'brand',
 'brand:wikidata',
 'brand:wikipedia',
 'name:fr',
 'payment:cash',
 'payment:debit_cards',
 'payment:credit_cards',
 'brand:en',
 'brand:fr',
 'official_name',
 'official_name:en',
 'official_name:fr',
 'payment:visa',
 'payment:mastercard',
 'payment:coins']